Pedro Nader
Assignment 9
This project tries to create a model that predicts if a tennis player will win his match or not.
Since points won is  a major predictor, it also does a regression to predict how many points a player will win

I know from experience that by far the most important factor in determining the winner is the total points won. So we will try to create a linear regression to determine total points won.

Problem Definition: We are using linear regression, and decision trees to try to predict wether or not a tennis player will win his game based on the stats.

Model Evaluation: Not a single model was enough. We had to combine models to accurately predict a win. The decision tree and logistic regression shows that the most important variable is percentage of points won, so we craeted a linear regression that predicts the percentage of points won based on other percentages, and then we can use the output (prediction) of the linear regression as input for the decision tree.

Final remarks: In summary, combining multiple models was a good idea. I also had to manipulate the data in a way that doesn't account for total serves or returns in but instead accounts for percentages.

In [379]:
#Importing everything
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics



In [380]:
#Saving stats into df
stats = pd.read_json('./Data/match_stats_2017_unindexed_json.json')

In [381]:
stats.columns  #Using to decisde which columns to use


Index(['loser_aces', 'loser_break_points_converted',
       'loser_break_points_return_total', 'loser_break_points_saved',
       'loser_break_points_serve_total', 'loser_double_faults',
       'loser_first_serve_points_total', 'loser_first_serve_points_won',
       'loser_first_serve_return_total', 'loser_first_serve_return_won',
       'loser_first_serves_in', 'loser_first_serves_total',
       'loser_return_games_played', 'loser_return_points_total',
       'loser_return_points_won', 'loser_second_serve_points_total',
       'loser_second_serve_points_won', 'loser_second_serve_return_total',
       'loser_second_serve_return_won', 'loser_service_games_played',
       'loser_service_points_total', 'loser_service_points_won',
       'loser_total_points_total', 'loser_total_points_won', 'match_duration',
       'match_id', 'match_stats_url_suffix', 'match_time', 'tourney_order',
       'winner_aces', 'winner_break_points_converted',
       'winner_break_points_return_total', 'winner_br

In [382]:
#Creating a sub DF that I will use for my model
#I first break it down into winner and loser DF then I concatenate them
modelDf = pd.DataFrame()

winnerDf = pd.DataFrame()
winnerDf['total_points_won'] = stats['winner_total_points_won']
winnerDf['total_point_played'] = stats['winner_total_points_total']
winnerDf['first_serve_points_won'] = stats['winner_first_serve_points_won']
winnerDf['first_serve_points_played'] = stats['winner_first_serve_points_total']
winnerDf['return_points_won'] = stats['winner_return_points_won']
winnerDf['return_points_played'] = stats['winner_return_points_total']
winnerDf['first_serves_in'] = stats['winner_first_serves_in']
winnerDf['first_serves_served'] = stats['winner_first_serves_total']
winnerDf['won'] = 1


loserDf = pd.DataFrame()
loserDf['total_points_won'] = stats['loser_total_points_won']
loserDf['total_point_played'] = stats['loser_total_points_total']
loserDf['first_serve_points_won'] = stats['loser_first_serve_points_won']
loserDf['first_serve_points_played'] = stats['loser_first_serve_points_total']
loserDf['return_points_won'] = stats['loser_return_points_won']
loserDf['return_points_played'] = stats['loser_return_points_total']
loserDf['first_serves_in'] = stats['loser_first_serves_in']
loserDf['first_serves_served'] = stats['loser_first_serves_total']
loserDf['won'] = 0



modelDf = pd.concat([winnerDf,loserDf]) #Since they have the same column names, they will be stacked on top of each other
modelDf

,total_points_won,total_point_played,first_serve_points_won,first_serve_points_played,return_points_won,return_points_played,first_serves_in,first_serves_served,won
0,97,181,50,62,31,84,62,97,1
1,77,146,41,52,24,69,52,77,1
2,66,119,27,36,21,61,36,58,1
3,75,138,37,56,24,61,56,77,1
4,94,176,42,52,29,82,52,94,1
...,...,...,...,...,...,...,...,...,...
3806,38,104,20,32,13,47,32,57,0
3807,86,183,41,67,27,79,67,104,0
3808,102,205,49,64,35,104,64,101,0
3809,28,84,16,34,8,36,34,48,0


In [383]:
#Notice how I created Column for percentages
modelDf['percent_points_won'] = modelDf['total_points_won'] / modelDf['total_point_played']
modelDf['percent_first_serve_points_won'] = modelDf['first_serve_points_won'] / modelDf['first_serve_points_played']
modelDf['percent_return_points_won'] = modelDf['return_points_won'] / modelDf['return_points_played']
modelDf['percent_first_serves_in'] = modelDf['first_serves_in'] / modelDf['first_serves_served']
modelDf.dropna()

modelDf = modelDf[(modelDf[['total_points_won','total_point_played','first_serve_points_won','first_serve_points_played','return_points_won','return_points_played','first_serves_in','first_serves_served','percent_points_won','percent_first_serve_points_won','percent_return_points_won','percent_first_serves_in']] != 0).all(axis=1)]



modelDf

,total_points_won,total_point_played,first_serve_points_won,first_serve_points_played,return_points_won,return_points_played,first_serves_in,first_serves_served,won,percent_points_won,percent_first_serve_points_won,percent_return_points_won,percent_first_serves_in
0,97,181,50,62,31,84,62,97,1,0.535912,0.806452,0.369048,0.639175
1,77,146,41,52,24,69,52,77,1,0.527397,0.788462,0.347826,0.675325
2,66,119,27,36,21,61,36,58,1,0.554622,0.750000,0.344262,0.620690
3,75,138,37,56,24,61,56,77,1,0.543478,0.660714,0.393443,0.727273
4,94,176,42,52,29,82,52,94,1,0.534091,0.807692,0.353659,0.553191
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3806,38,104,20,32,13,47,32,57,0,0.365385,0.625000,0.276596,0.561404
3807,86,183,41,67,27,79,67,104,0,0.469945,0.611940,0.341772,0.644231
3808,102,205,49,64,35,104,64,101,0,0.497561,0.765625,0.336538,0.633663
3809,28,84,16,34,8,36,34,48,0,0.333333,0.470588,0.222222,0.708333


In [384]:
#First create Decision tree with the only input being percentage of points won.
logX = modelDf[['percent_points_won']]
logY = modelDf['won']

In [385]:
logX_train, logX_test, logY_train, logY_test = train_test_split(logX, logY, test_size = 0.2, random_state = 0)

In [386]:
classifier = LogisticRegression(random_state=0)
classifier.fit(logX_train,logY_train)

LogisticRegression(random_state=0)

In [387]:
logY_pred = classifier.predict(logX_test)
logY_pred_prob = classifier.predict_proba(logX_test)
print(logY_pred)
print(logY_pred_prob)


[1 0 0 ... 0 1 1]
[[0.00500296 0.99499704]
 [0.84842603 0.15157397]
 [0.83117909 0.16882091]
 ...
 [0.88603619 0.11396381]
 [0.20493156 0.79506844]
 [0.0579347  0.9420653 ]]


In [388]:
print('The model can predict if the player will win or lose with  ' + str(accuracy_score(logY_pred, logY_test)*100) +'% accuracy')

The model can predict if the player will win or lose with  94.68503937007874% accuracy


In [389]:
#Confusion matrix to test for false positive or false negatives
idx = ['actual 0', 'actual 1']
cm = pd.DataFrame(confusion_matrix(logY_test, logY_pred),
                  columns = ['pred 0', 'pred 1'],
                  index= idx)
cm

,pred 0,pred 1
actual 0,718,48
actual 1,33,725


#As you can see, the percent of points won is an incredibly good predictor of a win. 90%+ accuracy

We will now create a regression that predicts percentage of points won

In [390]:
regressionX = modelDf[['percent_first_serve_points_won','percent_return_points_won','percent_first_serves_in']]
regressionY = modelDf['percent_points_won']

In [391]:
regressionX_train, regressionX_test, regressionY_train, regressionY_test = train_test_split(regressionX, regressionY, test_size = 0.2, random_state = 0)

In [392]:
my_regression = LinearRegression()
my_regression.fit(regressionX_train,regressionY_train)

LinearRegression()

In [393]:
#Printing R squared
print('R^2 = {}'.format(my_regression.score(regressionX_train,regressionY_train)))

R^2 = 0.8882626007871377


In [394]:
#Getting all the coeficients to define the regression equation. 
print('The intercept (b_0) is  {}'.format(my_regression.intercept_))
print('The coef (b_1) is  {}'.format(my_regression.coef_[0]))
print('The coef (b_2) is  {}'.format(my_regression.coef_[1]))

The intercept (b_0) is  0.0071357162752371806
The coef (b_1) is  0.33303331926993496
The coef (b_2) is  0.4978331156750676


In [395]:
#Saving all the predictions to evaluate performance after
regression_prediction = my_regression.predict(regressionX_test)

In [396]:
#Evaluating the model
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(regressionY_test, regression_prediction))
print('Mean Squared Error:', metrics.mean_squared_error(regressionY_test, regression_prediction))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(regressionY_test, regression_prediction)))

Mean Absolute Error: 0.01702092430758223
Mean Squared Error: 0.0004938738509342379
Root Mean Squared Error: 0.022223272732301107


In [397]:
#Only a mean absolute error of 1%! Meaning our model is pretty good

In [398]:
#I will try to create a function that tests for the medium error
#This is different beacuase it is calculating the average percetage.
# I could use mean absolute error to calculate the absolute points it gets wrong.
def error(expected,predicted):
        l1 = list(expected)
        l2 = list(predicted)
        errors = []
        for i in range(len(l1)):
                thiserror = abs(l1[i] - l2[i])
                errors.append(thiserror/l1[i])
        return np.mean(errors)

In [399]:
print('on average the results are off by {}'.format(str(error(regressionY_test,regression_prediction)*100)) + "%")

on average the results are off by 3.5027518520324494%


Our regression model works very well to predict percentage of points won! Meaning we have a very good way to predict if the player will win or not!

The last step is to combine both models

I will create a function for this

In [400]:
def predictWin(independentVariablesDF): #Input has to be a pd dataFrame
    #Run same thing as before
    percentPointsPred = my_regression.predict(independentVariablesDF)
    logY_pred = classifier.predict(pd.DataFrame(data=percentPointsPred,columns=['percent_points_won']))
    logY_pred_prob = classifier.predict_proba(pd.DataFrame(data=percentPointsPred,columns=['percent_points_won']))
    return [logY_pred,logY_pred_prob]

In [401]:
#applying to real life
finalPred = predictWin(regressionX_test) # With this stats I should win a game
print(finalPred) #This gives us the probability of winning
print(len(finalPred[0]))
print(len(finalPred[1]))



[array([1, 0, 0, ..., 0, 1, 1]), array([[0.0122054 , 0.9877946 ],
       [0.91474383, 0.08525617],
       [0.91667117, 0.08332883],
       ...,
       [0.76404478, 0.23595522],
       [0.3079888 , 0.6920112 ],
       [0.09904085, 0.90095915]])]
1524
1524


In [402]:
#Finally, I want to calculate the final accuracy

In [403]:
print('The model can predict if the player will win or lose with  ' + str(accuracy_score(finalPred[0], logY_test)*100) +'% accuracy')

The model can predict if the player will win or lose with  89.10761154855643% accuracy


In [404]:
predictWin(pd.DataFrame([[0.5,0.6,0.5]])) # This is a real world test from some of my games #Predicts as a win with 74% chance of winning

/Users/pedronader/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


[array([1]), array([[0.2588459, 0.7411541]])]